#  Introduction

## Problem Statement
*  there is a change in trend that occurs due to uncertain conditions causing excessive stock of goods, so sometimes goods that have been purchased cannot be resold because the product has been piling up too long in the inventory.

*  determining the storage of stock goods by analyzing using sales data for the previous month / year because the data in the previous month usually has an impact on future sales, while sales that occur do not always follow the previous month's data 

## Data Provided
The Data that is used in this project are
*     **sale_items.csv** - Contains the details of historical daily sales of a customer
*     **sales.csv** - Contains the historical daily sales of a customer
*     **sma_categories.csv** - Contains info about category of products
*     **sma_companies.csv** - Contains information about the customer of company X
*     **sma_products.csv** - Contains list of products that is sold in the companies
We will have a sneak peak into the dataset below 

# Content:
1. Data Preprocessing
     - Read the dataset
     - Drop null value and several columns
     - join dataset into 1 dataset
2. Exploratory Data Analysis
3. Search correlation between data
4. Implement model
     - use only data sales 
     - use with data external such as IHK, Inflasi, IHP
5. Use specific data for trend product
6. Use Time lag in analysis

In [1]:
#load library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os #using operating system dependent functionality
from datetime import datetime#datetime module supplies classes for manipulating dates and times.
import math # provides access to the mathematical functions
from IPython.display import display, HTML

#For Plotting
# Using plotly + cufflinks in offline mode
import plotly as py
import plotly.graph_objs as go
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)
init_notebook_mode(connected=True)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#For time series decomposition
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose

from datetime import timedelta

#Pandas option
pd.options.display.float_format = '{:.2f}'.format

# 1. Data Preprocessing

## 1.1 Read Dataset

In [2]:
#read dataset
df_sales = pd.read_csv('proyek_skripsi/sales.csv')
df_sale_items = pd.read_csv('proyek_skripsi/sale_items.csv')
df_products = pd.read_csv('proyek_skripsi/sma_products.csv')
df_category = pd.read_csv('proyek_skripsi/sma_categories.csv')
df_companies = pd.read_csv('proyek_skripsi/sma_companies.csv')
df_ihk_inflasi = pd.read_csv("proyek_skripsi/Data_IHK_Inflasi_Nasional.csv")
df_ihp = pd.read_csv("proyek_skripsi/DataIHPKhususPlastikKaret.csv")

C:\Users\USER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.

C:\Users\USER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.



## 1.2 Clean Dataset

In [3]:
def clean_data(dropped_column, df, negative_column = []):
    df = df.drop(dropped_column, axis=1)
    df = df[(df[negative_column] >= 0).all(1)]
    #df = df.fillna(0)
    df = df.dropna()
    return df

In [4]:
dropped_sales = ['note', 'staff_note', 'order_discount_id', 'updated_by', 'updated_at', 'return_id', 
                 'attachment', 'return_sale_ref', 'sale_id', 'rounding', 'suspend_note', 'address_id', 
                 'reserve_id', 'salesman_id', 'salesman_commission', 'total_commission', 'product_discount', 
                 'total_discount', 'grand_total', 'payment_term', 'due_date', 'paid', 'customer', 'quote_id', 
                 'reference_no', 'biller_id', 'biller', 'warehouse_id', 'order_discount', 'product_tax', 
                 'order_tax_id', 'order_tax', 'total_tax', 'shipping', 'pos', 'surcharge', 'api', 'shop', 'hash', 
                 'created_by', 'sale_status', 'payment_status', 'return_sale_total', 'total_items']

df_sales = clean_data(dropped_sales, df_sales, ['total'])
df_sales = df_sales.rename(columns={'id': 'sale_id'})
df_sales = df_sales.loc[(df_sales['date'] >= '2017-02-31')]
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17392 entries, 0 to 17496
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sale_id      17392 non-null  int64  
 1   date         17392 non-null  object 
 2   customer_id  17392 non-null  int64  
 3   total        17392 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 679.4+ KB


In [5]:
dropped_saleitems = ['serial_no', 'sale_item_id', 'comment', 'id', 'product_type', 'option_id', 'unit_price', 
                     'net_unit_price', 'warehouse_id', 'item_tax', 'tax_rate_id', 'tax', 'product_unit_id', 
                     'product_unit_code', 'unit_quantity']

df_sale_items = clean_data(dropped_saleitems, df_sale_items, ['quantity', 'item_discount', 'real_unit_price', 'subtotal'])
df_sale_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52393 entries, 5 to 52594
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sale_id          52393 non-null  int64  
 1   product_id       52393 non-null  int64  
 2   product_code     52393 non-null  object 
 3   product_name     52393 non-null  object 
 4   quantity         52393 non-null  float64
 5   discount         52393 non-null  object 
 6   item_discount    52393 non-null  float64
 7   subtotal         52393 non-null  float64
 8   real_unit_price  52393 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 4.0+ MB


In [6]:
dropped_product = ['code', 'name', 'unit', 'product_price', 'alert_quantity', 'image', 'subcategory_id', 'cf1', 'cf2', 'cf3', 'cf4',
       'cf5', 'cf6', 'quantity', 'tax_rate', 'track_quantity', 'details',
       'warehouse', 'barcode_symbology', 'file', 'product_details',
       'tax_method', 'type', 'supplier1', 'supplier1price', 'supplier2',
       'supplier2price', 'supplier3', 'supplier3price', 'supplier4',
       'supplier4price', 'supplier5', 'supplier5price', 'promotion',
       'promo_price', 'start_date', 'end_date', 'supplier1_part_no',
       'supplier2_part_no', 'supplier3_part_no', 'supplier4_part_no',
       'supplier5_part_no', 'sale_unit', 'purchase_unit', 'brand', 'slug',
       'featured', 'weight']

df_products = clean_data(dropped_product, df_products)
df_products = df_products.rename(columns={'id': 'product_id'})
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5746 entries, 0 to 5745
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product_id   5746 non-null   int64  
 1   cost         5746 non-null   float64
 2   category_id  5746 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 179.6 KB


In [7]:
dropped_category = ['image', 'parent_id', 'slug']

df_category = clean_data(dropped_category, df_category)
df_category = df_category.rename(columns={'id': 'category_id', 'code': 'category_code', 'name': 'category_name'})
df_category.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category_id    40 non-null     int64 
 1   category_code  40 non-null     object
 2   category_name  40 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


In [8]:
dropped_companies = ['group_id', 'vat_no', 'group_name', 'customer_group_id',
       'customer_group_name', 'name', 'address', 'state', 'postal_code', 'country', 'phone', 'email', 'cf1', 'cf2',
       'cf3', 'cf4', 'cf5', 'cf6', 'invoice_footer', 'payment_term', 'logo',
       'award_points', 'deposit_amount', 'price_group_id', 'price_group_name',
       'email_footer']

df_companies = clean_data(dropped_companies, df_companies)
df_companies = df_companies.rename(columns={'id': 'customer_id'})
df_companies = df_companies[~df_companies.city.str.contains("-")]
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441 entries, 0 to 454
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  441 non-null    int64 
 1   company      441 non-null    object
 2   city         441 non-null    object
dtypes: int64(1), object(2)
memory usage: 13.8+ KB


### make variant of location into one location

In [9]:
#change list into Jawa
list_jawa = ['JAWA TIMUR', 'BALI', 'SURABAYA', 'MADURA', 'JAWA TENGAH', 'JAKARTA', 'DENPASAR', 'JAWA TINUR', 'SBY', 'PARE-KEDIRI, JATIM', 'BAU BAU', 'BONDOWOSO', 
    'GRESIK', 'TULUNGAGUNG, JAWA TIMUR', 'LOMBOK', 'JAWA TENGGAH', 'BLITAR, JAWA TIMUR',
    'JOGJA', '( PERUM DARMO SENTOSA RAYA) ,SURABAYA', 'JAKARTA BARAT', 'BLORA, JAWA TENGAH'
    'PASURUAN, JAWA TIMUR', 'BOJONEGORO', 'JAWA BARAT', 'GENTENG', 'CEPU', 'KEDIRI, JAWA TIMUR', 'SURABAYA, JATIM', 'MALANG']

for i in list_jawa:
    df_companies.loc[df_companies['city'].str.contains(i), 'city'] = 'JAWA'

    
#change list into sulawesi
list_sulawesi = ["BANGGAI, SULAWESI", "SLAWESI", "WAKATOBI, SULAWESI", "MAKASAR"]

for i in list_sulawesi:
    df_companies.loc[df_companies['city'].str.contains(i), 'city'] = 'SULAWESI'

    
#change list into papua
list_papua = ["JAYAPURA", "MERAUKE"]

for i in list_papua:
    df_companies.loc[df_companies['city'].str.contains(i), 'city'] = 'PAPUA'


#change list into kalimantan
df_companies.loc[df_companies['city'].str.contains(" KALIMANTAN"), 'city'] = 'KALIMANTAN'


#change list into nusa tenggara
list_nusatenggara = ["NTT", "NTB"]

for i in list_nusatenggara:
    df_companies.loc[df_companies['city'].str.contains(i), 'city'] = 'NUSA TENGGARA'


df_companies['city'].value_counts()

C:\Users\USER\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



JAWA             201
SULAWESI         126
KALIMANTAN        67
PAPUA             33
NUSA TENGGARA     13
SUMATRA            1
Name: city, dtype: int64